# Problem Description

#### Background Research
According to bbc.com, London-focused estate agent Foxtons has swung to a loss and says the housing market in the capital is 
in a prolonged downturn.Foxtons said annual revenues fell 5% to £111.5m, with the weakness in property sales being offset 
slightly by a resilient lettings performance.Measures of consumer confidence weakened around the turn of the year and 
surveyors reported a further fall in new buyer enquiries over the same period.
While the number of properties coming onto the market also slowed, this doesn't appear to have been enough to prevent a modest shift in the balance of demand and supply in favour of buyers in recent months.

#### Business Problem
To recommend the Buyers location of real estate in London with market value of the assest for wise investement in this adverse condition.


# Data Description

Data required for this problem is London properties details and the relative price paid data. Data was extracted from the HM Land Registry.Address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name.House number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.
We will use Clustering technique once the neigborhood data is extracted using Foursqure API, this will help in understanding the locations with good amenities housing property in reasonable price.

We will select only London City data and will try to get the average housing price street wise in London City. Finally we will plot the recommended location in London map.

# Methodology

##### Import the libraries

In [3]:
import pandas as pd
import os 
import numpy as np
import datetime as dt 
import json
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  48.48 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.47 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.37 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  48.11 MB/s
Libraries imported.


In [4]:
# Get the data from HM Land Registry website
df = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

In [6]:
#Explore the data
df.head()
print(df.shape)
#We need to perform pre-processing on the data
# Passing the column names to the dataframe
df.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON','SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']
df.describe(include='object')
df.head()
#checking the dataset info


(993670, 16)


,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{726BF13A-993F-0A46-E053-6C04A8C01D0D},115000,2018-07-13 00:00,DL17 9LB,S,N,F,4,NaN,THE LANE,WEST CORNFORTH,FERRYHILL,COUNTY DURHAM,COUNTY DURHAM,A,A
1,{726BF13A-9940-0A46-E053-6C04A8C01D0D},24000,2018-04-10 00:00,SR7 9AG,F,N,L,20B,NaN,WOODS TERRACE,MURTON,SEAHAM,COUNTY DURHAM,COUNTY DURHAM,A,A
2,{726BF13A-9941-0A46-E053-6C04A8C01D0D},56000,2018-06-22 00:00,DL5 5PS,T,N,F,6,NaN,HEILD CLOSE,NaN,NEWTON AYCLIFFE,COUNTY DURHAM,COUNTY DURHAM,A,A
3,{726BF13A-9942-0A46-E053-6C04A8C01D0D},220000,2018-05-25 00:00,DL16 7HE,D,N,F,25,NaN,BECKWITH CLOSE,KIRK MERRINGTON,SPENNYMOOR,COUNTY DURHAM,COUNTY DURHAM,A,A
4,{726BF13A-9943-0A46-E053-6C04A8C01D0D},58000,2018-05-09 00:00,DL14 6FH,F,N,L,23,NaN,AINTREE DRIVE,NaN,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM,A,A


In [7]:
# As we are intreseted to check the London City info, we will create a dataframe and save only London City data
df_london=df[df['Town_City']=='LONDON']
print(df_london.shape)
df_london.head()


(63692, 16)


,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
225,{666758D6-BF44-3363-E053-6B04A8C0D74E},470000,2018-02-09 00:00,E4 7RL,S,N,F,10,NaN,BUTLERS DRIVE,NaN,LONDON,EPPING FOREST,ESSEX,A,A
262,{666758D6-B8E7-3363-E053-6B04A8C0D74E},675000,2018-02-07 00:00,E2 0PW,F,N,L,BOW BROOK HOUSE,FLAT 29,GATHORNE STREET,NaN,LONDON,TOWER HAMLETS,GREATER LONDON,A,A
264,{666758D6-B8EA-3363-E053-6B04A8C0D74E},284000,2018-01-03 00:00,E15 4HW,F,N,L,35B,NaN,GOVIER CLOSE,NaN,LONDON,NEWHAM,GREATER LONDON,A,A
266,{666758D6-B8EC-3363-E053-6B04A8C0D74E},337500,2018-01-12 00:00,N17 9DJ,F,N,L,187,NaN,BREAM CLOSE,NaN,LONDON,HARINGEY,GREATER LONDON,A,A
270,{666758D6-B8F0-3363-E053-6B04A8C0D74E},575000,2018-01-16 00:00,N4 4NL,F,N,L,24C,NaN,OAKFIELD ROAD,NaN,LONDON,HARINGEY,GREATER LONDON,A,A


In [8]:
#Convert the Date_Transfer column in standard date format
df_london.Date_Transfer=df_london.Date_Transfer.apply(pd.to_datetime)
# Delete all obsolete transactions which were done before 2016
df_london.drop(df_london[df_london.Date_Transfer.dt.year < 2016].index, inplace=True)
#sort by date of sale
df_london.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)
df_london.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
917831,{8355F009-DACE-55C5-E053-6B04A8C0D090},1095000,2018-12-31,SW8 4EU,F,Y,L,"BROGAN HOUSE, 9",APARTMENT 76,ST JOSEPHS STREET,NaN,LONDON,WANDSWORTH,GREATER LONDON,B,A
230383,{8355F009-40A4-55C5-E053-6B04A8C0D090},1070000,2018-12-31,N1C 4PF,F,Y,L,98,FLAT 44,CAMLEY STREET,NaN,LONDON,CAMDEN,GREATER LONDON,A,A
247859,{8355F009-6070-55C5-E053-6B04A8C0D090},370000,2018-12-31,SE25 6TX,T,N,F,13,NaN,BROSTER GARDENS,NaN,LONDON,CROYDON,GREATER LONDON,A,A
917830,{8355F009-DACD-55C5-E053-6B04A8C0D090},715000,2018-12-31,SW8 4EU,F,Y,L,"BROGAN HOUSE, 9",APARTMENT 75,ST JOSEPHS STREET,NaN,LONDON,WANDSWORTH,GREATER LONDON,B,A
353735,{80E1AA99-1207-7BF8-E053-6C04A8C00BF2},3139950,2018-12-31,SW1H 0HY,O,N,L,ARTILLERY MANSIONS,FLAT 82,VICTORIA STREET,NaN,LONDON,CITY OF WESTMINSTER,GREATER LONDON,B,A


In [9]:
#Get the average price of the housing asset streetwise in London
# list of street names in LONDON
streets = df_london['Street'].unique().tolist()
df_grp_pr = df_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_pr.columns = ['Street', 'Avg_Price']

In [11]:
#Now we want to get the streets which fall under affordabale price
#first define the range of affordable price
df_budget=df_grp_pr[(df_grp_pr['Avg_Price']>=3000000)&(df_grp_pr['Avg_Price']<=3750000)]
print(df_budget.shape)
df_budget.head()

(143, 2)


,Street,Avg_Price
13,ABBEY TRADING POINT,3.000000e+06
23,ABBOTSBURY ROAD,3.361250e+06
160,ALBANY,3.250000e+06
522,ARUNDEL STREET,3.559933e+06
889,BASKERVILLE ROAD,3.462500e+06


In [13]:
# We need to get the lattitude and longitude of streets
geolocator = Nominatim()
df_budget['street_cord'] = df_budget['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df_budget

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Street,Avg_Price,street_cord
13,ABBEY TRADING POINT,3.000000e+06,"(49.18179205, -2.07997384418976)"
23,ABBOTSBURY ROAD,3.361250e+06,"(51.3954954, -0.1949358)"
160,ALBANY,3.250000e+06,"(42.6511674, -73.754968)"
522,ARUNDEL STREET,3.559933e+06,"(51.5126526, -0.1145121)"
889,BASKERVILLE ROAD,3.462500e+06,"(51.4494512, -0.1704881)"
904,BATHGATE ROAD,3.172000e+06,"(51.436445, -0.2190543)"
1440,BOURDON STREET,3.531383e+06,"(53.4881904, -2.2154327)"
1609,BREWERY SQUARE,3.300000e+06,"(51.52387345, -0.102042039207886)"
1732,BROMPTON SQUARE,3.650000e+06,"(51.4980849, -0.1685937)"
1884,BULL INN COURT,3.150000e+06,"(-37.6110806, 143.7209297)"


In [14]:
df_budget[['Latitude', 'Longitude']] = df_budget['street_cord'].apply(pd.Series)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/frame.py:2540: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [15]:
df_street_cords=df_budget.drop(columns=['street_cord'])
df_street_cords.head()

,Street,Avg_Price,Latitude,Longitude
13,ABBEY TRADING POINT,3.000000e+06,49.181792,-2.079974
23,ABBOTSBURY ROAD,3.361250e+06,51.395495,-0.194936
160,ALBANY,3.250000e+06,42.651167,-73.754968
522,ARUNDEL STREET,3.559933e+06,51.512653,-0.114512
889,BASKERVILLE ROAD,3.462500e+06,51.449451,-0.170488


In [64]:
df_street_cords.shape

(143, 4)

In [16]:
#create the map of these streets on London
#London's lat, long from google
latitude=51.5074
longitude=-0.1278

map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, price, street in zip(df_street_cords['Latitude'], df_street_cords['Longitude'], df_street_cords['Avg_Price'], df_street_cords['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

## Foursquare

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
# Now we will explore the venues near by the streets in our dataframe where the real estate property are in the declared budget
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df_street_cords['Street'],
                                   latitudes=df_street_cords['Latitude'],
                                   longitudes=df_street_cords['Longitude']
                                  )

ABBEY TRADING POINT
ABBOTSBURY ROAD
ALBANY
ARUNDEL STREET
BASKERVILLE ROAD
BATHGATE ROAD
BOURDON STREET
BREWERY SQUARE
BROMPTON SQUARE
BULL INN COURT
CADOGAN SQUARE
CADOGAN STREET
CALVIN STREET
CAMDEN PARK ROAD
CAMPDEN GROVE
CAMPDEN STREET
CANONBURY SQUARE
CENTRE AVENUE
CHAGFORD STREET
CHALCOT ROAD
CHARLES II PLACE
CHESTER STREET
CHEYNE GARDENS
CHILTERN STREET
CHURCH WALK
CLAPHAM COMMON WEST SIDE
CLARENDON PLACE
CLAREVILLE STREET
CLEVELAND STREET
CLIFTON HILL
CLIVEDEN PLACE
CLOCKHOUSE CLOSE
CLORANE GARDENS
COPPICE DRIVE
CORNHILL
COURTNELL STREET
COWCROSS STREET
DEALTRY ROAD
DENBIGH TERRACE
DENYER STREET
DEVONSHIRE MEWS SOUTH
DORLCOTE ROAD
DRAYTON GARDENS
DUNRAVEN STREET
ELLERBY STREET
ENNISMORE GARDENS
FIRST STREET
FOURNIER STREET
FRANKLINS ROW
FREEMASONS ROAD
FROGNAL RISE
GLENGALL ROAD
GLENTHAM ROAD
GOODHART PLACE
GRANGE GARDENS
GREAT NEWPORT STREET
GREAT WEST ROAD
GREEN ROAD
GREENWICH PARK STREET
GROVELANDS ROAD
HALSEY STREET
HAMMERSMITH TERRACE
HARDESS STREET
HARRIS STREET
HARTFIELD

In [73]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ABBEY TRADING POINT,49.181792,-2.079974,Longueville Manor,49.184102,-2.079418,Hotel
1,ABBEY TRADING POINT,49.181792,-2.079974,Waitrose & Partners,49.179651,-2.084373,Supermarket
2,ABBEY TRADING POINT,49.181792,-2.079974,Caddy Snax,49.178999,-2.081043,Food Truck
3,ABBOTSBURY ROAD,51.395495,-0.194936,Mr Cod,51.396980,-0.190786,Fast Food Restaurant
4,ABBOTSBURY ROAD,51.395495,-0.194936,Sainsbury's Local,51.395157,-0.194113,Convenience Store
5,ABBOTSBURY ROAD,51.395495,-0.194936,Princess of India,51.399830,-0.196827,Indian Restaurant
6,ABBOTSBURY ROAD,51.395495,-0.194936,Domino's Pizza,51.399581,-0.196854,Pizza Place
7,ABBOTSBURY ROAD,51.395495,-0.194936,Morden South Railway Station (MDS),51.393758,-0.200108,Train Station
8,ALBANY,42.651167,-73.754968,Renaissance Albany Hotel,42.650625,-73.755687,Hotel
9,ALBANY,42.651167,-73.754968,City Beer Hall,42.649660,-73.754787,Pub


In [21]:
location_venues.groupby('Street').count()
location_venues.head()

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ABBEY TRADING POINT,49.181792,-2.079974,Longueville Manor,49.184102,-2.079418,Hotel
1,ABBEY TRADING POINT,49.181792,-2.079974,Waitrose & Partners,49.179651,-2.084373,Supermarket
2,ABBEY TRADING POINT,49.181792,-2.079974,Caddy Snax,49.178999,-2.081043,Food Truck
3,ABBOTSBURY ROAD,51.395495,-0.194936,Mr Cod,51.396980,-0.190786,Fast Food Restaurant
4,ABBOTSBURY ROAD,51.395495,-0.194936,Sainsbury's Local,51.395157,-0.194113,Convenience Store


In [22]:
#One hot encoding for Venue Category
#Street column and Venue Category dummies will be used to create new dataframe
venue_dummi = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venue_dummi['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_col = [venue_dummi.columns[-1]] + list(venue_dummi.columns[:-1])

#fixed_columns
venue_dummi = venue_dummi[fixed_col]

print(venue_dummi.shape)
venue_dummi.head()

(5970, 341)


,Street,Accessories Store,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,...,Warehouse Store,Whisky Bar,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant
0,ABBEY TRADING POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ABBEY TRADING POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ABBEY TRADING POINT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ABBOTSBURY ROAD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ABBOTSBURY ROAD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#We will check the Venue Categories available street-wise by group by on street
street_venue_grp=venue_dummi.groupby('Street').mean().reset_index()
print(street_venue_grp.shape)
street_venue_grp.head()

(134, 341)


,Street,Accessories Store,African Restaurant,Airport,Airport Service,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Arcade,...,Warehouse Store,Whisky Bar,Windmill,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Yoshoku Restaurant
0,ABBEY TRADING POINT,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,ABBOTSBURY ROAD,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,ALBANY,0.0,0.0,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,ARUNDEL STREET,0.0,0.0,0.0,0.0,0.022727,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.011364,0.011364,0.0,0.0,0.0,0.0,0.0
4,BASKERVILLE ROAD,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0


In [24]:
#Top5 venues in real estates in the budget
top5_venues = 5

for hood in street_venue_grp['Street']:
    print("----"+hood+"----")
    temp = street_venue_grp[street_venue_grp['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top5_venues))
    print('\n')

----ABBEY TRADING POINT----
               venue  freq
0        Supermarket  0.33
1              Hotel  0.33
2         Food Truck  0.33
3  Accessories Store  0.00
4        Pastry Shop  0.00


----ABBOTSBURY ROAD----
                  venue  freq
0         Train Station   0.2
1     Convenience Store   0.2
2  Fast Food Restaurant   0.2
3           Pizza Place   0.2
4     Indian Restaurant   0.2


----ALBANY----
         venue  freq
0         Café  0.11
1          Pub  0.08
2  Coffee Shop  0.08
3        Hotel  0.08
4   Restaurant  0.05


----ARUNDEL STREET----
          venue  freq
0           Pub  0.08
1       Theater  0.07
2   Coffee Shop  0.07
3         Hotel  0.06
4  Cocktail Bar  0.06


----BASKERVILLE ROAD----
               venue  freq
0               Café  0.25
1                Pub  0.25
2  French Restaurant  0.12
3      Garden Center  0.12
4  Convenience Store  0.12


----BATHGATE ROAD----
            venue  freq
0  Tennis Stadium  0.18
1    Tennis Court  0.18
2          Lounge  

In [25]:
#function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:top_venues]

In [26]:
top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))



In [28]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = street_venue_grp['Street']

for ind in np.arange(street_venue_grp.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(street_venue_grp.iloc[ind, :], top_venues)



In [29]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ABBEY TRADING POINT,Hotel,Supermarket,Food Truck,Yoshoku Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop
1,ABBOTSBURY ROAD,Train Station,Pizza Place,Indian Restaurant,Convenience Store,Fast Food Restaurant,Yoshoku Restaurant,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market
2,ALBANY,Café,Hotel,Coffee Shop,Pub,Pizza Place,Bar,Restaurant,Park,American Restaurant,Food Truck
3,ARUNDEL STREET,Pub,Theater,Coffee Shop,Hotel,Cocktail Bar,Sandwich Place,Restaurant,Italian Restaurant,Japanese Restaurant,French Restaurant
4,BASKERVILLE ROAD,Pub,Café,Garden Center,Tennis Court,French Restaurant,Convenience Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop


In [30]:
print(venues_sorted.shape)
print(street_venue_grp.shape)

(134, 11)
(134, 341)


In [31]:
#df_street_cords
street_venue_grp=df_street_cords
street_venue_grp.head()

,Street,Avg_Price,Latitude,Longitude
13,ABBEY TRADING POINT,3.000000e+06,49.181792,-2.079974
23,ABBOTSBURY ROAD,3.361250e+06,51.395495,-0.194936
160,ALBANY,3.250000e+06,42.651167,-73.754968
522,ARUNDEL STREET,3.559933e+06,51.512653,-0.114512
889,BASKERVILLE ROAD,3.462500e+06,51.449451,-0.170488


## Clustering

In [32]:
#We will cluster the dataframe based on venues/amenities/facilities near  by.
# set number of clusters
kclust = 5

london_clustering = street_venue_grp.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclust, random_state=0).fit(london_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]


array([1, 0, 4, 3, 0, 4, 3, 4, 2, 4, 1, 3, 2, 3, 3, 1, 4, 0, 2, 4, 1, 4, 0,
       3, 0, 0, 3, 3, 1, 3, 4, 0, 1, 3, 1, 0, 0, 0, 0, 4, 3, 1, 1, 1, 1, 3,
       4, 3, 1, 1], dtype=int32)

In [33]:
#Dataframe to include Clusters

london_grouped_clustering=df_street_cords
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
13,ABBEY TRADING POINT,3.000000e+06,49.181792,-2.079974
23,ABBOTSBURY ROAD,3.361250e+06,51.395495,-0.194936
160,ALBANY,3.250000e+06,42.651167,-73.754968
522,ARUNDEL STREET,3.559933e+06,51.512653,-0.114512
889,BASKERVILLE ROAD,3.462500e+06,51.449451,-0.170488


In [34]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,ABBEY TRADING POINT,3.000000e+06,49.181792,-2.079974,1,Hotel,Supermarket,Food Truck,Yoshoku Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop
23,ABBOTSBURY ROAD,3.361250e+06,51.395495,-0.194936,0,Train Station,Pizza Place,Indian Restaurant,Convenience Store,Fast Food Restaurant,Yoshoku Restaurant,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market
160,ALBANY,3.250000e+06,42.651167,-73.754968,4,Café,Hotel,Coffee Shop,Pub,Pizza Place,Bar,Restaurant,Park,American Restaurant,Food Truck
522,ARUNDEL STREET,3.559933e+06,51.512653,-0.114512,3,Pub,Theater,Coffee Shop,Hotel,Cocktail Bar,Sandwich Place,Restaurant,Italian Restaurant,Japanese Restaurant,French Restaurant
889,BASKERVILLE ROAD,3.462500e+06,51.449451,-0.170488,0,Pub,Café,Garden Center,Tennis Court,French Restaurant,Convenience Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
904,BATHGATE ROAD,3.172000e+06,51.436445,-0.219054,4,Tennis Stadium,Tennis Court,Scenic Lookout,Bus Stop,Restaurant,Museum,Souvenir Shop,Lounge,Food Court,Yoshoku Restaurant
1440,BOURDON STREET,3.531383e+06,53.488190,-2.215433,3,Department Store,Park,Gym Pool,Food Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court
1609,BREWERY SQUARE,3.300000e+06,51.523873,-0.102042,4,Pub,Coffee Shop,Hotel,Café,Italian Restaurant,Plaza,Steakhouse,Vietnamese Restaurant,Bar,Sushi Restaurant
1732,BROMPTON SQUARE,3.650000e+06,51.498085,-0.168594,2,Café,Italian Restaurant,Hotel,Boutique,Exhibit,Coffee Shop,Science Museum,Chinese Restaurant,Tea Room,Cocktail Bar
1884,BULL INN COURT,3.150000e+06,-37.611081,143.720930,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclust)
ys = [i+x+(i*x)**2 for i in range(kclust)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [36]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,3361250.0,Train Station,Pizza Place,Indian Restaurant,Convenience Store,Fast Food Restaurant,Yoshoku Restaurant,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market
889,3462500.0,Pub,Café,Garden Center,Tennis Court,French Restaurant,Convenience Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
2364,3400000.0,Pizza Place,Ice Cream Shop,Dry Cleaner,Bagel Shop,Convenience Store,Train Station,Chinese Restaurant,Thai Restaurant,Spa,Gym
2613,3380335.0,Coffee Shop,Train Station,Sports Club,Cricket Ground,Yoshoku Restaurant,Food Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
2711,3371900.0,Pub,Indian Restaurant,Department Store,Bar,Accessories Store,Bus Station,Thrift / Vintage Store,Grocery Store,Greek Restaurant,Brewery


In [37]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,3.000000e+06,Hotel,Supermarket,Food Truck,Yoshoku Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop
2016,3.119000e+06,Café,Hotel,Italian Restaurant,Boutique,Japanese Restaurant,Restaurant,Pub,Clothing Store,Cocktail Bar,Sporting Goods Shop
2110,3.118417e+06,Pub,Bakery,Indian Restaurant,Hotel,Bookstore,Italian Restaurant,Yoga Studio,Ice Cream Shop,Grocery Store,Restaurant
2459,3.025000e+06,Bakery,Hotel,English Restaurant,Pizza Place,Clothing Store,Café,Bookstore,Japanese Restaurant,Art Gallery,Italian Restaurant
2829,3.047017e+06,Coffee Shop,Restaurant,Café,Japanese Restaurant,Italian Restaurant,Hotel,Cocktail Bar,Pizza Place,French Restaurant,Thai Restaurant


In [38]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1732,3650000.0,Café,Italian Restaurant,Hotel,Boutique,Exhibit,Coffee Shop,Science Museum,Chinese Restaurant,Tea Room,Cocktail Bar
2053,3650000.0,Café,Indian Restaurant,Clothing Store,Flea Market,Pub,Food Truck,Coffee Shop,Salon / Barbershop,Pizza Place,Vegetarian / Vegan Restaurant
2376,3750000.0,Café,Pub,Thai Restaurant,Hotel,Museum,Coffee Shop,Pizza Place,Movie Theater,Gym / Fitness Center,Sandwich Place
5023,3662500.0,Café,Pub,Italian Restaurant,Bakery,Pizza Place,Ice Cream Shop,Japanese Restaurant,Clothing Store,Coffee Shop,Creperie
5784,3650000.0,Italian Restaurant,Indian Restaurant,Grocery Store,Gastropub,Pub,Café,Thai Restaurant,Middle Eastern Restaurant,Greek Restaurant,English Restaurant


In [39]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
522,3.559933e+06,Pub,Theater,Coffee Shop,Hotel,Cocktail Bar,Sandwich Place,Restaurant,Italian Restaurant,Japanese Restaurant,French Restaurant
1440,3.531383e+06,Department Store,Park,Gym Pool,Food Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court
2017,3.500000e+06,Pharmacy,Brewery,Pizza Place,Park,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
2090,3.500000e+06,Park,Grocery Store,Deli / Bodega,Pub,Convenience Store,Pizza Place,Event Space,Liquor Store,Turkish Restaurant,Food & Drink Shop
2103,3.488750e+06,Pub,Clothing Store,Juice Bar,Restaurant,Japanese Restaurant,Art Gallery,Grocery Store,Garden,French Restaurant,Outdoor Supply Store


In [40]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()


,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
160,3250000.0,Café,Hotel,Coffee Shop,Pub,Pizza Place,Bar,Restaurant,Park,American Restaurant,Food Truck
904,3172000.0,Tennis Stadium,Tennis Court,Scenic Lookout,Bus Stop,Restaurant,Museum,Souvenir Shop,Lounge,Food Court,Yoshoku Restaurant
1609,3300000.0,Pub,Coffee Shop,Hotel,Café,Italian Restaurant,Plaza,Steakhouse,Vietnamese Restaurant,Bar,Sushi Restaurant
1884,3150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2152,3155000.0,Café,Pub,Bakery,Music Venue,Italian Restaurant,Cocktail Bar,Coffee Shop,Mediterranean Restaurant,Furniture / Home Store,Turkish Restaurant


# Results and Discussions

Even though the London Housing Market may be in downfall, There is still much opportunity and scope in it.

We have analyzed the data in two aspects, one analysis is based on the affordable prices grouped by street/neighborhood areas and another is by clustering the locations based on the venues and amenities.

Anaysis on Neigborhood areas in London:
It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Analysis by Clusters based on amenities and venues near by:
Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

# Conclusion

To conclude, The housing market in the capital(London) is in a prolonged downturn  It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, spaorts club, restaurant, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.
